This notebook serves as a sandbox for testing different hyperparameters for training the neural network. While this notebook can  be ran without a Nvidia GPU, the usage of one is reccomended as this notebook was written with using CUDA when running PyTorch in mind.

In [1]:
#Import neccessary libraries
import numpy as np
import pandas as pd
import torch as th

In [47]:
#Read data from dataset.csv and convert into numpy arrays
data_frame = pd.read_csv("dataset.csv")
data_array = data_frame.to_numpy()
labels = data_array[:, 34].astype(str)
features = np.delete(data_array, 34, 1)

#Convert strings in labels array into ints and set as type 'int64'
#Additionally, for the purposes of this project, data that has the label 'Enrolled' will be ignored
int_labels = np.empty(3630, dtype = 'float32')
remove_list = []
label_index = 0
for i in range(4424):
    if(labels[i] == "Dropout"):
        int_labels[label_index] = 0
        label_index += 1
    elif(labels[i] == "Graduate"):
        int_labels[label_index] = 1
        label_index += 1
    elif(labels[i] == "Enrolled"):
        remove_list.append(i)
labels = int_labels
features = np.delete(features, remove_list, 0)

#Clean up features array by removing biased features, one-hot encoding, and standardization
features = np.delete(features, [0, 1, 2, 6, 7, 8, 9, 10, 11, 12, 15, 18], 1)

course_encode = np.zeros((3630, 17), dtype = 'float32')
quali_encode = np.zeros((3630, 17), dtype = 'float32')
for i in range(3630):
    course = features[i, 0]
    course_encode[i, (course - 1)] = 1
    quali = features[i, 2]
    quali_encode[i, (quali - 1)] = 1
hot_features = np.concatenate((course_encode, quali_encode), axis = 1)

bool_features = np.vstack((features[:, 1], features[:, 3], features[:, 4], features[:, 5])).astype('float32').T
unstd_features = np.delete(features, [0, 1, 2, 3, 4, 5], 1).astype('float32')
std_features = np.empty((3630, 16), dtype = 'float32')
for i in range(16):
    mean = np.mean(unstd_features[:, i])
    std = np.std(unstd_features[:, i])
    for j in range(3630):
        std_features[j, i] = (unstd_features[j, i] - mean) / std

features = np.concatenate((std_features, bool_features, hot_features), axis = 1)

#Dataset class used for creating datasets out of the training and validation data
class Dataset(th.utils.data.Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.features[idx, :], self.labels[idx]
    
#Function for creating dataloaders using the training and validation sets made with getSets
#The dataloader passes samples in batches of 66
def build_loaders(tfeatures, tlabels, vfeatures, vlabels):
    tdataset = Dataset(tfeatures, tlabels)
    vdataset = Dataset(vfeatures, vlabels)
    tloader = th.utils.data.DataLoader(tdataset, batch_size = 66)
    vloader = th.utils.data.DataLoader(vdataset, batch_size = 66)
    return tloader, vloader

#NeuralNetwork class used for the building the model
#Neural Network sepcific hyperparameters are edited directly in this class
class NeuralNetwork(th.nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = th.nn.Flatten()
        self.fc1 = th.nn.Linear(54 , 27)
        self.drop = th.nn.Dropout(0.2)
        self.fc2 = th.nn.Linear(27, 1)
        
    def forward(self, x):
        x = self.flatten(x)
        x = th.nn.functional.relu(self.fc1(x))
        x = self.drop(x)
        x = self.fc2(x)
        return x
    
#Select the device to be used for training the model and send the model to it
#Uses GPU if a Nvdia GPU is detected and PyTorch was installed with the CUDA platform, uses CPU otherwise
device = "cuda" if th.cuda.is_available() else "cpu"
model = NeuralNetwork().to(device)

#Define a loss function and an optimizer for training the model
loss_func = th.nn.BCEWithLogitsLoss()
optimizer = th.optim.SGD(model.parameters(), lr = 0.0007, momentum = 0.99, weight_decay = 0.01)

#Define training function
def train(tloader, model, loss_func, optimizer):
    size = len(tloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(tloader):
        X, y = X.to(device), y.to(device)
        prediction = model(X)
        prediction = prediction.squeeze()
        loss = loss_func(prediction, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

#Define testing fuction
def test(vloader, model, loss_func):
    size = len(vloader.dataset)
    numBatches = len(vloader)
    model.eval()
    correct = 0
    with th.no_grad():
        for X, y in vloader:
            X, y = X.to(device), y.to(device)
            prediction = model(X)
            prediction = prediction.squeeze()
            prediction = th.sigmoid(prediction)
            prediction = (prediction > 0.5).type(th.float)
            correct += (prediction == y).type(th.float).sum().item()
    correct /= size
    print(f"Test Accuracy: {(100 * correct):>0.1f}%\n")

#Create datasets for trianing data and validation data
pindex = np.random.permutation(3630)
vnum = int(3630 * 0.2)
vindex = pindex[:vnum]
tindex = pindex[vnum:]
tfeatures = features[tindex]
tlabels = labels[tindex]
vfeatures = features[vindex]
vlabels = labels[vindex]

#Train the model for the given number of epochs and print out the training and testing error
epochs = 20
tloader, vloader = build_loaders(tfeatures, tlabels, vfeatures, vlabels)
for i in range(epochs):
    print(f"Epoch {i + 1}\n-------------------------------")
    train(tloader, model, loss_func, optimizer)
    test(vloader, model, loss_func)
    
#Save the model and model weights to model.pth and model_weights.pth 
th.save(model.state_dict(), 'model_weights.pth')
th.save(model, 'model.pth')

Epoch 1
-------------------------------
Test Accuracy: 80.9%

Epoch 2
-------------------------------
Test Accuracy: 76.0%

Epoch 3
-------------------------------
Test Accuracy: 78.0%

Epoch 4
-------------------------------
Test Accuracy: 81.3%

Epoch 5
-------------------------------
Test Accuracy: 85.7%

Epoch 6
-------------------------------
Test Accuracy: 87.3%

Epoch 7
-------------------------------
Test Accuracy: 87.9%

Epoch 8
-------------------------------
Test Accuracy: 87.7%

Epoch 9
-------------------------------
Test Accuracy: 88.8%

Epoch 10
-------------------------------
Test Accuracy: 89.7%

Epoch 11
-------------------------------
Test Accuracy: 89.3%

Epoch 12
-------------------------------
Test Accuracy: 89.4%

Epoch 13
-------------------------------
Test Accuracy: 89.1%

Epoch 14
-------------------------------
Test Accuracy: 88.8%

Epoch 15
-------------------------------
Test Accuracy: 89.5%

Epoch 16
-------------------------------
Test Accuracy: 90.2%

E

With the current model, validation accuracy is within the range of 90-91% accurate after 20 epochs of training.